In [1]:
import pandas as pd
import pickle
import re
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def = pd.concat([df,df2])

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

# Descargar los recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    if text is None:
        return ''
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
    # Tokenización
    tokens = nltk.word_tokenize(text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

descripcion_filtrada = df_def['Descripcion'].apply(preprocess_text)

print(descripcion_filtrada)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


https://www.indomio.es/anuncios/95227781/    alquila piso amueblado habitaciones carrer del...
https://www.indomio.es/anuncios/95146207/    piso estudiantes entrar partir ro febrero juni...
https://www.indomio.es/anuncios/94613177/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/94500757/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/95223135/    rincón acogedor necesito espacio gustará aquí ...
                                                                   ...                        
https://www.indomio.es/anuncios/95007509/    atico m valencia valencia solidinmuebles vende...
https://www.indomio.es/anuncios/94958481/    venden pisos obra nueva camino moncada trata r...
https://www.indomio.es/anuncios/95006805/    vende exclusiva viviendas obra nueva calle arq...
https://www.indomio.es/anuncios/95313313/    vende piso barrio tormoseste luminoso apartame...
https://www.indomio.es/anuncios/95319871/    piso 

## LSA

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

tv = TfidfVectorizer()
svd = TruncatedSVD(n_components=100)
lsa = make_pipeline(tv, svd, Normalizer(copy=False))
lsa_topic_vectors = lsa.fit_transform(descripcion_filtrada)

lsa_topic_vectors

array([[ 0.14551729, -0.27074119,  0.79033064, ...,  0.01639877,
         0.04020484,  0.05724126],
       [ 0.19709968, -0.20329222,  0.03446927, ..., -0.00519293,
         0.02348342,  0.00619282],
       [ 0.28958638, -0.31155587, -0.08265641, ..., -0.06236709,
         0.06293022, -0.10907441],
       ...,
       [ 0.25436477, -0.23142527, -0.06158474, ..., -0.08479562,
        -0.0681465 , -0.14314343],
       [ 0.40881147, -0.39530807,  0.00324198, ..., -0.13959552,
         0.05539602,  0.05180022],
       [ 0.30516304, -0.32295559,  0.01396056, ..., -0.04398804,
         0.22464639,  0.05794318]])

In [4]:
# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción usando la misma pipeline LSA
vector_descripcion = lsa.transform([descripcion])

# Calcular la similitud coseno entre la nueva descripción y todas las descripciones existentes
similarities = cosine_similarity(vector_descripcion, lsa_topic_vectors)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Opcionalmente, puedes mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()


Descripción: GRAN OPORTUNIDAD DE INVERSIÓN EN PISO DE ESTUDIANTES La vivienda está recién reformada, ubicada a menos de 500m de una de las mejores playas de Valencia, así como a 700m de la Universidad Politécnica de Valencia. Cuenta con la parada de tranvía justo en frente, así como supermercados, bancos, restaurantes y Universidades. El barrio del Cabanyal en Poblats Maritims, es perfecto para invertir, ya que se trata de un punto intermedio entre la zona universitaria y la zona turística; pudiendo de esta forma lograr un alquiler durante el período del curso escolar, y turístico en los meses de verano. La vivienda cuenta con 76m2, reforma intregral y equipamiento en cada uno de los espacios. Las 4 habitaciones están amuebladas (2 de ellas exteriores); 2 baños completos con plato de ducha e hidromasaje. Cocina equipada con todos los electrodomésticos básicos, además de lavavajillas y Ósmosis (purificador de agua). Se trata de una segunda planta sin ascensor
Similitud: 0.52495090372204

## LDA

In [5]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)

tf = tf_vectorizer.fit_transform(descripcion_filtrada)

lda = LatentDirichletAllocation(n_components=10, max_iter=5,
learning_method='online',
learning_offset=50.,
random_state=0)

lda_topic_matrix = lda.fit_transform(tf)

In [6]:
# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción
vector_descripcion = tf_vectorizer.transform([descripcion])
vector_descripcion_lda = lda.transform(vector_descripcion)

# Calcular la similitud coseno entre la nueva descripción y las descripciones existentes
similarities = cosine_similarity(vector_descripcion_lda, lda_topic_matrix)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()

Descripción: Bonito piso recién reformado. Está todo a estrenar! A tan solo unos minutos de la plaza del Ayuntamienti y del Mercado Central. 
Similitud: 0.9999964260327223

Descripción: Piso en pleno centro de Valencia de 545 m2 con terrazas muy grandes. Dispone de un apartamento añadido para los invitados.el piso es de estilo señorial con acabados de primera calidad reformado y a estrenar.
Similitud: 0.9999261534660971

Descripción: Apartamento de 3 dormitorios en una excelente ubicación, Restaurantes, supermercados cercas.El dormitorio principal tiene un vestidor y un baño en suite. La cocina está amueblada con área de lavandería separada. Aire acondicionado
Similitud: 0.9999139048645552

Descripción: Un encantador apartamento con lindas paredes azules en la sala de estar, una espaciosa cocina y una gran terraza en un vecindario tranquilo y residencial. El apartamento está a solo 10 minutos a pie del puerto de Valencia, y cerca encontrará algunas tiendas, una piscina municipal y una 

### Comprobar la coherencia de los topics LDA

In [7]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

# Paso 1: Preparar datos para gensim
# Convertir el texto tokenizado (asumiendo que tienes 'descripcion_filtrada' como una lista de listas de palabras)
texts = [doc.split() for doc in descripcion_filtrada]

# Crear un diccionario de Gensim a partir de los textos
dictionary = Dictionary(texts)

# Crear un corpus de Gensim utilizando el diccionario
corpus = [dictionary.doc2bow(text) for text in texts]

# Paso 2: Extraer los tópicos del modelo LDA de scikit-learn
topics = []
feature_names = tf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    topics.append([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])

# Paso 3: Calcular la coherencia del modelo
cm = CoherenceModel(model=None, topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("Coherencia del modelo LDA:", coherence)


Coherencia del modelo LDA: 0.7025929061753893


## NMF

In [8]:
# Inicialización de CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
tf = tf_vectorizer.fit_transform(descripcion_filtrada)

# Uso de NMF
nmf = NMF(n_components=10, random_state=0, max_iter=300)

# Transformar los datos con NMF
nmf_topic_matrix = nmf.fit_transform(tf)

In [9]:
# Nueva descripción a comparar
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción usando el mismo vectorizador
vector_descripcion = tf_vectorizer.transform([descripcion])
vector_descripcion_nmf = nmf.transform(vector_descripcion)

# Calcular la similitud coseno entre la nueva descripción y las descripciones existentes transformadas
similarities = cosine_similarity(vector_descripcion_nmf, nmf_topic_matrix)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()

Descripción: SE ALQUILA PISO (ZONA DE MALILLA) Piso nuevo Edificio emblemático, frente al nuevo Hospital de la Fe, Piso de 63M2 moderno ,paredes lisas ,cocina office ,balcón ,armarios empotrados con todos los servicios transporte publico, zonas verdes, supermercados y todo el equipamiento necesario. Vivienda nueva ideal para una/dos personas, luminosa, alegre, acogedora, tranquila y totalmente amueblada. Agua caliente y frio/calor por aerotermia. Puerta acorazada, ventanales con aislamiento completo. (garaje incluido) CONSULTANOS PARA MAS INFORMACION INMOCITY
Similitud: 0.9771943584682952

Descripción: SE VENDE EXCLUSIVO PISO . SEMI NUEVO En la zona nueva de Nazaret junto a las torres y aun paso de la playa. Piso De altas calidades en RESIDENCIAL con zonas comunes Piscina pádel zona para niños. El piso se distribuye en 84M2 y 100M2 construidos . Fantástico comedor con salida a la terraza con cerramiento de cristales en forma de laminas y correderos . cocina independiente 3 habitaciones

### Comprobar la coherencia de los topics NMF

In [10]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

# Paso 1: Preparar datos para gensim
texts = [doc.split() for doc in descripcion_filtrada]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Paso 2: Extraer los tópicos del modelo NMF
topics = []
feature_names = tf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf.components_):
    top_features_ind = topic.argsort()[:-10 - 1:-1]  # obtener los índices de las palabras más importantes
    topic_words = [feature_names[i] for i in top_features_ind]
    topics.append(topic_words)

# Paso 3: Calcular la coherencia del modelo
cm = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("Coherencia del modelo NMF:", coherence)


Coherencia del modelo NMF: 0.7463658663036883


### Mostrar palabras clave de cada tema en NMF

In [11]:
# Obtener las palabras del vocabulario
feature_names = tf_vectorizer.get_feature_names_out()

# Función para mostrar las palabras clave por tema
def show_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx + 1}:")
        # Muestra las n_top_words palabras más importantes por tema
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        top_features_str = ", ".join(top_features)
        print(f"Palabras clave: {top_features_str}\n")

# Mostrar las palabras clave de cada tema
n_top_words = 10
show_top_words(nmf, feature_names, n_top_words)

Topic #1:
Palabras clave: datos, tratamiento, servicios, consentimiento, fin, mantener, personales, hogarabitat, sl, mismos

Topic #2:
Palabras clave: vivienda, cuenta, cocina, baño, salón, toda, gran, aire, dormitorios, dispone

Topic #3:
Palabras clave: piso, habitaciones, cocina, ascensor, balcón, exterior, luminoso, cuenta, cerca, baños

Topic #4:
Palabras clave: the, and, with, room, of, patio, to, in, is, interior

Topic #5:
Palabras clave: valencia, ciudad, barrio, centro, plaza, gran, calle, ayuntamiento, mercado, situado

Topic #6:
Palabras clave: espacio, hogar, ofrece, disfrutar, cuenta, comodidad, vida, cada, oportunidad, además

Topic #7:
Palabras clave: zona, metro, gran, ma, toda, bien, equipada, salida, tipo, transporte

Topic #8:
Palabras clave: planta, terraza, do, cocina, acceso, casa, baño, primera, garaje, vista

Topic #9:
Palabras clave: inmueble, precio, anuncio, puede, informativo, contractual, vinculante, valencia, muestra, errores

Topic #10:
Palabras clave: a

## HDI

In [16]:
import pandas as pd
from gensim import similarities
from gensim.corpora import Dictionary
from gensim.models import HdpModel
from gensim.matutils import sparse2full

# Crear un diccionario de características y filtrar palabras extremadamente frecuentes o infrecuentes
texts = [doc.split() for doc in descripcion_filtrada]
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.95)

# Crear el corpus BOW
corpus = [dictionary.doc2bow(text) for text in texts]

# Entrenar el modelo HDI
hdi = HdpModel(corpus, id2word=dictionary)

# Función para obtener la distribución de tópicos de un documento
def get_topic_distribution(text):
    bow = dictionary.doc2bow(text.lower().split())
    topic_sparse = hdi[bow]
    return sparse2full(topic_sparse, len(hdi.id2word))

# Convertir todos los documentos a vectores de tópicos
topic_vectors = [get_topic_distribution(text) for text in descripcion_filtrada]

In [18]:
# Nueva descripción
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud coseno entre la nueva descripción y todas las descripciones existentes
sims = cosine_similarity([descripcion_vec], topic_vectors)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = sims.argsort()[0][::-1]

# Opcionalmente, muestra las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {sims[0][index]}")
    print()

Descripción: piso en venta ubicado en calle escultor vicente beltran grimal, valència. superficie de 61 m2 .
Similitud: 0.9999999403953552

Descripción: Se alquila piso amueblado de dos habitaciones en la C/ De Quart. La vivienda está en la segunda planta de un edificio sin ascensor.
Similitud: 0.9999990463256836

Descripción: atentos la vivienda se vende con ocupantes ilegales no se puede visitar, son los antiguos inquilinos con el contrato vencido sin comisiones
Similitud: 0.9999988675117493

Descripción: Piso para entrar de inmediato, totalmente reformado y amueblado para entrar a vivir, está decorado y equipado para entrar de inmediato, 
Similitud: 0.9999988675117493

Descripción: Se alquila piso de 2 habitaciones amueblado en Carrer Dels Serrans. La propiedad incluye balcón y aire acondicionado.
Similitud: 0.9999988079071045

